In [1]:
from os import listdir
from os.path import isfile, join

mypath = "/lakehouse/default/Files/silver"

for item in listdir(mypath): 
    if(item.startswith("dim_")): 
        df_silver = (
            spark.read
             .format("delta")
            .load("Files/silver/"+item)
            )
        df_silver.where(df_silver.status ==1) \
                .select("Code", "Description")\
                .write.mode("overwrite")\
                .format("delta")\
                .save("Files/gold/"+item)

StatementMeta(, aecc74af-4b00-482e-8e56-48dbe1e57d2b, 3, Finished, Available)

In [2]:
#change this to a for loop for all the facts like that of dims 

mypath = "/lakehouse/default/Files/gold"

for item in listdir(mypath): 
    if(item.startswith("dim_")): 
        df_gold = (
            spark.read
            .format("delta")
            .load("Files/gold/"+item)
            )
        spark.sql("DROP TABLE IF EXISTS "+item)
        df_gold.createOrReplaceTempView("VW_"+item)
        spark.sql("CREATE TABLE IF NOT EXISTS "+item+" AS SELECT * FROM VW_"+item)


StatementMeta(, aecc74af-4b00-482e-8e56-48dbe1e57d2b, 4, Finished, Available)

In [3]:
mypath = "/lakehouse/default/Files/gold"

df_fact = (spark.read.format("delta").load("Files/silver/fact_monthly_inter_invest"))
 
df_refarea = (spark.read.format("delta").load("Files/gold/dim_area"))
df_counterarea = (spark.read.format("delta").load("Files/gold/dim_area")) 
df_flowstock = (spark.read.format("delta").load("Files/gold/dim_flow_stock_entry")) 
df_accounting = (spark.read.format("delta").load("Files/gold/dim_accounting")) 
df_accitem = (spark.read.format("delta").load("Files/gold/dim_accounts_item"))
df_maturity = (spark.read.format("delta").load("Files/gold/dim_maturity"))
df_unit = (spark.read.format("delta").load("Files/gold/dim_unit_measure"))
df_currency = (spark.read.format("delta").load("Files/gold/dim_currency"))
df_val = (spark.read.format("delta").load("Files/gold/dim_valuation"))
df_comp = (spark.read.format("delta").load("Files/gold/dim_comp_method"))
df_type = (spark.read.format("delta").load("Files/gold/dim_type_entity"))
df_refsector = (spark.read.format("delta").load("Files/gold/dim_sector"))
df_countersector = (spark.read.format("delta").load("Files/gold/dim_sector"))

StatementMeta(, aecc74af-4b00-482e-8e56-48dbe1e57d2b, 5, Finished, Available)

In [4]:

df_fact_gold = df_fact.join(df_counterarea, df_fact.COUNTERPART_AREA == df_counterarea.Code,'inner').withColumn("COUNTERPART_AREA", df_counterarea.Description)\
        .join(df_refarea, df_fact.REF_AREA == df_refarea.Code,'inner').withColumn("REF_AREA", df_refarea.Description)\
        .join(df_flowstock, df_fact.FLOW_STOCK_ENTRY == df_flowstock.Code,'inner').withColumn("FLOW_STOCK_ENTRY", df_flowstock.Description)\
        .join(df_accounting, df_fact.ACCOUNTING_ENTRY == df_accounting.Code,'inner').withColumn("ACCOUNTING", df_accounting.Description)\
        .join(df_accitem, df_fact.INT_ACC_ITEM == df_accitem.Code,'inner').withColumn("ACCOUNTS_ITEM", df_accitem.Description)\
        .join(df_maturity, df_fact.MATURITY == df_maturity.Code,'inner').withColumn("MATURITY", df_maturity.Description)\
        .join(df_unit, df_fact.UNIT_MEASURE == df_unit.Code,'inner').withColumn("UNIT_MEASURE", df_unit.Description)\
        .join(df_currency, df_fact.CURRENCY_DENOM == df_currency.Code,'inner').withColumn("CURRENCY", df_currency.Description)\
        .join(df_val, df_fact.VALUATION == df_val.Code,'inner').withColumn("VALUATION", df_val.Description)\
        .join(df_comp, df_fact.COMP_METHOD == df_comp.Code,'inner').withColumn("COMP_METHOD", df_comp.Description)\
        .join(df_type, df_fact.TYPE_ENTITY == df_type.Code,'inner').withColumn("TYPE_ENTITY", df_type.Description)\
        .join(df_refsector, df_fact.REF_SECTOR == df_refsector.Code,'inner').withColumn("REF_SECTOR", df_refsector.Description)\
        .join(df_countersector, df_fact.COUNTERPART_SECTOR == df_countersector.Code,'inner').withColumn("COUNTERPART_SECTOR", df_countersector.Description)\
        .select("TITLE", "FUNCTIONAL_CAT","TIME_PERIOD","OBS_VALUE" ,"OBS_STATUS","REF_PERIOD_DETAIL", "REF_AREA", "REF_SECTOR", "COUNTERPART_AREA", "COUNTERPART_SECTOR","FLOW_STOCK_ENTRY", "ACCOUNTING", "ACCOUNTS_ITEM", "MATURITY","UNIT_MEASURE", "CURRENCY", "VALUATION","COMP_METHOD","TYPE_ENTITY")

StatementMeta(, aecc74af-4b00-482e-8e56-48dbe1e57d2b, 6, Finished, Available)

In [5]:
df_fact_gold.write.mode("overwrite")\
                .format("delta")\
                .save("Files/gold/fact_monthly_inter_invest")

spark.sql("DROP TABLE IF EXISTS fact_monthly_inter_invest")
df_fact_gold.createOrReplaceTempView("VW_fact_monthly_inter_invest")
spark.sql("CREATE TABLE IF NOT EXISTS fact_monthly_inter_invest AS SELECT * FROM VW_fact_monthly_inter_invest")

StatementMeta(, aecc74af-4b00-482e-8e56-48dbe1e57d2b, 7, Finished, Available)

DataFrame[]

In [ ]:
%%sql
-- M.REF_AREA = REFAREA.CODE
--AND M.COUNTERPART_AREA = COUNTERAREA.CODE
  --  AND M.FLOW_STOCK_ENTRY = FLOWSTOCK.CODE 
   -- AND M.ACCOUNTING_ENTRY = ACCOUNTING.CODE
 --   AND M.INT_ACC_ITEM = ACCITEM.CODE 
   -- AND M.MATURTIY = MAT.CODE 
   -- AND M.UNIT_MEASURE = UNIT.CODE 
  --  AND M.CURRENCY_DENOM = CURRENCY.CODE 
 --   AND M.VALUATION = VAL.CODE 
   -- AND M.COMP_METHOD = COMP.CODE 
   -- AND M.TYPE_ENTITY = TYPEENT.CODE
    --AND M.REF_SECTOR = REFSECTOR.CODE 
  --  AND M.COUNTERPAR_SECTOR = COUNTERSECTOR.CODE

   -- M.FUNCTIONAL_CAT , 
 --   M.TIME_PERIOD , 
 --   M.OBS_VALUE , 
 --   M.OBS_STATUS , 
 --   M.COMPILING_ORG , 
  --  M.REF_PERIOD_DETAIL , 
  --  M.TITLE 